In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# Function that runs the simulation

duration = 200 # total time in ms
refractory_period = 5  # refractory period in ms
def LIF(tau=5, threshold=0.8, reset=0.1, spike_height=0.8):
    # List of tuples with (spike time, weight)
    time_weight_pairs = [
        (20, 0.4),  
        (40, 0.78),  
        (60, 0.05), 
        (80, 0.65), 
        (100, 0.42), 
        (120, 0.7),
        (140, 0.1), 
        (160, 0.35),
        (180, 0.05),
    ]    
    time_weight_pairs.sort(key=lambda pair: pair[0], reverse=True)

    # Rest of the initializations remain the same
    dt = 0.1  # timestep in ms
    alpha = np.exp(-dt / tau)  # decay factor
    V_rec = []  # list to record membrane potentials
    V = 0.0  # initial membrane potential
    T = np.arange(np.round(duration / dt)) * dt  # array of times
    spikes = []  # list to store spike times

    # run the simulation
    for t in T:
        V_rec.append(V)  # record
        V *= alpha  # integrate equations
        while time_weight_pairs and t >= time_weight_pairs[-1][0]:
            _, weight = time_weight_pairs.pop()
            V += weight

        if V > threshold:
            spikes.append(t)
            V_rec.append(spike_height)  # Record the spike height
            V = reset  # then reset
        else:
            V_rec.append(V)  # Record V if no spike

    return T, V_rec, time_weight_pairs, spikes

# Update function for the animation to include smooth transitions

def update(frame):
    # Define the parameters to change over time

    # Calculate which time point we are at in the animation
    current_time = frame * duration / num_frames
    threshold = 0.8
    reset = 0.0

    # Run the LIF model with the current parameters up to the current_time
    T, V_rec, time_weight_pairs, spikes = LIF(tau=5, threshold=threshold, reset=reset)

    # Ensure we only take the portion of T and V_rec up to the current frame
    # Here, we assume that for each time in T_frame, there are two entries in V_rec
    # One before the potential update and one after
    T_frame = T[T <= current_time]
    V_rec_frame = V_rec[:len(T_frame) * 2]  # This needs to be checked if it is correct

    # Correct the times for spikes to reflect the times we have spikes in the system
    times_frame = [t for t, _ in time_weight_pairs if t <= current_time]

    # Correct the spikes_frame to contain only spikes up to the current time
    spikes_frame = [s for s in spikes if s <= current_time]

    # Update the plot
    ax.clear()
    # Plot vertical lines for input spikes
    for t in times_frame:
        ax.axvline(t, ls=':', c='b')
    
    # Plot the membrane potential
    ax.plot(np.repeat(T_frame, 2)[:len(V_rec_frame)], V_rec_frame, '-k', lw=2)

    # Plot vertical lines for output spikes
    for s in spikes_frame:
        ax.axvline(s, ls='--', c='r', label='spike')
    ax.axhline(threshold, ls='--', c='g', label='threshold')
    ax.set_xlim(0, duration)
    ax.set_ylim(-1, 2)
    ax.set_xlabel('Time [ms]')
    ax.set_ylabel('Membrane Potential [V]')
    ax.legend(loc='lower right')
    
    # Text display of the current tau and weight values
    ax.text(0.95, 0.95, f'time: {current_time:.2f} ms', horizontalalignment='right',
            verticalalignment='top', transform=ax.transAxes, color='blue')



# Set up the figure for the animation
fig, ax = plt.subplots(figsize=(5, 4))
plt.close(fig)  # Close the figure to prevent it from showing now

# Number of frames in the animation
num_frames = duration

# Create the animation
ani = FuncAnimation(fig, update, frames=num_frames, blit=False)

# Save the animation
ani.save('lif_neuron_simulation5.gif', writer='imagemagick', fps=20)  # Increase fps for a faster animation